In [1]:
import torch
import torch.nn as nn
import pandas as pd

import sys
sys.path.append('../')
from HGP.models import LightModel
import pickle

In [2]:
import networkx as nx

In [3]:
from utils.imports import *
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import SimpleProfiler

# BITTE EINMAL DEN MODELL NAMEN HIER EINMAL EINTRAGEN (HGP-SL o.ä.)

Damit die Datei unter dem Namen gespeichert wird

In [ ]:
model_name = "HGPSL_Enzymes"

In [7]:
# %%
args_dict = take_hp("../configs/emp-enzym-equal.yml")

pl.seed_everything(args_dict["random_seed"], workers=True)


Global seed set to 42


42

In [8]:
# %%
# Load the dataset
dataset = TUDataset(
    Path.cwd().joinpath("../data"),
    name=args_dict["dataset"],
    use_node_attr=True,
)
args_dict["num_classes"] = dataset.num_classes
args_dict["num_features"] = dataset.num_features

In [9]:
split_train = int(dataset.len() * args_dict["split_ratio"])
split_val = int((dataset.len() - split_train) * args_dict["test_ratio"])
split_test = dataset.len() - split_val - split_train

print(
    f"Hole: {dataset.len()}, Train: {split_train}, Val: {split_val}, Test: {split_test}"
)
# %%
train_data, eval_data, test_data = random_split(
    dataset,
    [split_train, split_val, split_test],
    generator=torch.Generator().manual_seed(42),
)

Hole: 600, Train: 420, Val: 90, Test: 90


train_data, eval_data, test_data = random_split(
    dataset,
    [split_train, split_val, split_test],
    generator=torch.Generator().manual_seed(42),
)

In [10]:
# %%
train_loader = DataLoader(
    train_data,
    batch_size=args_dict["batch_size"],
    shuffle=False,
    num_workers=args_dict["num_workers"],
)
val_loader = DataLoader(
    eval_data,
    batch_size=args_dict["batch_size"],
    shuffle=False,
    num_workers=args_dict["num_workers"],
)
test_loader = DataLoader(
    test_data,
    batch_size= 1 ,# dataset.len() - split_test,
    shuffle=False,
    num_workers=args_dict["num_workers"],
)

In [71]:
# Loop over all models in the model/emp-enzym-equal folder
# and classify the graphs
loss = nn.CrossEntropyLoss(reduction="none")
softmax = nn.Softmax(dim=1)
Path(f"../analysis/{args_dict['architecture']}/{args_dict['dataset']}").mkdir(parents=True, exist_ok=True)


for i in range(5):
    model = LightModel(args_dict).to(args_dict["device"])
    model.eval()
    model.freeze()
    model.load_state_dict(torch.load(f"../model/emp-enzyme_equal/model_{i}.pt"))
    graph_ids = []

    graphs = []
    counter = 0
    for batch in test_loader:
        data = batch.to(args_dict["device"])
        out = model(data)
        prediction_truth = np.argmax(softmax(out).detach().cpu().numpy()) == data.y.detach().cpu().numpy()
        
        graph_dict = {

            "replication": f"{i+1}",	
            "graph_id": f"{counter}",
            "prediction": f"{np.argmax(softmax(out).detach().cpu().numpy())}",
            "label": f"{data.y.detach().cpu().numpy()[0]}",
            "correct": f"{prediction_truth[0]}",
            "confidence": softmax(out).detach().cpu().numpy()[0][np.argmax(softmax(out).detach().cpu().numpy())],
            "loss": loss(out, data.y).detach().cpu().numpy()[0]
        }
        
        g = nx.from_edgelist(data.edge_index.cpu().numpy().T)
        g.graph.update(graph_dict)
        graphs.append(g)
        
        counter += 1

    np_graphs = np.array(graphs, dtype=object)
    np.save(f"../analysis/{args_dict['architecture']}/{args_dict['dataset']}/graphs_{i}.npy", np_graphs)

In [72]:
graph_dict

{'replication': '5',
 'graph_id': '89',
 'prediction': '2',
 'label': '5',
 'correct': 'False',
 'confidence': 0.29514322,
 'loss': 3.2211344}